## Mid-level benchmarks

### Private Information Retrieval (PIR)

One application of HE is to retrieve a data point from a database held elsewhere, without the database holder knowing which point is being requested. If Alice wants to query Bob's database, she simply encrypts an array full of zeros, except for a "1" in the position of the desired data point, and sends this ciphertext to Bob. He then performs homomorphic multiplication and addition, and sends the ciphertext back to Alice, who decrypts it to retrieve the data point she requested.

To demonstrate this on a very small scale, lets look at a "database" containing only two values: 123 and 456:

In [1]:
from frontend.database import BenchmarkMeasurement, session
from frontend.database import build_filter
from scripts import generate_pir_circuit, generate_variance_circuit, generate_bitonic_sort_circuit, benchmark_utils

First let's generate the circuit file for this simple case:

In [4]:
circuit_file=generate_pir_circuit.generate(2,[2])

What inputs does this circuit expect?

In [5]:
benchmark_utils.get_inputs(circuit_file)

['d_0_0_0', 'd_0_1_0', 's_0_0', 's_0_1']

"d_a_b_c" are the "database" values, and "s_x_y" are the "selector" values.  We can set d_0_0_0 and d_0_1_0 to "123" and "456" respectively.  Then, to select d_0_0_0 (which will hopefully be the value "123") from the database, we should set s_0_0 to 1 and s_0_1 to zero.

In [6]:
inputs_file = benchmark_utils.write_inputs_file({"d_0_0_0": 123, "d_0_1_0": 456, "s_0_0": 1, "s_0_1": 0})

In [7]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"int8_t","HElib_Fp")

In [8]:
results


{'e_0_0': '123'}

Assuming we will normally want to query a database containing N>2 items, one might think that we need to encrypt and send N "s"-values to identify the data point that we want.  However, we can be smarter than that by using a binary tree structure for the data, and having the "s"-values dictate how we navigate the tree to locate the desired data point. 

Lets generate a circuit file corresponding to a database containing 32 values, arranged in a tree with 5 layers and 2 choices per layer ($2^5 = 32$).

In [15]:
circuit_file=generate_pir_circuit.generate(128,[2,2,2,2,2,2,2])

Let's now fill this database with values 0-to-31:

In [16]:
inputlist=benchmark_utils.get_inputs(circuit_file)
data = [(x, i) for (i, x) in enumerate(inputlist) if x.startswith('d_')]

To start with, we'll set all the "s" (selector) inputs to zero:

In [17]:
data += [(x, 0) for (i, x) in enumerate(inputlist) if x.startswith('s_')]
data = dict(data)


Now we need to set certain selector variables to "1" to navigate down the tree.  Suppose we want to choose the last element of the database (should be value "31") - we need to go down the right-hand branches:

In [18]:
data['s_0_1'] = 1
data['s_1_1'] = 1
data['s_2_1'] = 1
data['s_3_1'] = 1
data['s_4_1'] = 1
data['s_5_1'] = 1
data['s_6_1'] = 1

In [19]:
inputs_file = benchmark_utils.write_inputs_file(data)

In [24]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"int16_t","HElib_F2")

KeyboardInterrupt: 

In [23]:
results

{'e_6_0': '6895'}

## Calculating mean and variance of a set of inputs

One may wish to calculate statistical properties of a set of encrypted inputs, such as mean, standard deviation etc.   Currently the contexts implemented in SHEEP do not have "Divide" operations, so calculating these exact values via only homomorphic operations on the ciphertext is not possible.  

However, the client will necessarily know "N", the number of inputs, so can perform division in the clear on the decrypted results of the homomorphic calculations.

We therefore only need homomorphic addition and multiplication.  Simply summing the inputs $x_i$ gives us $N\bar{x}$. 
Meanwhile $\Sigma_{i=0}^N(Nx_i - N\bar{x})^2$  is $N^3$ times the variance.



Let's generate a circuit to calculate $(N \times mean)$ and ($N^3 \times variance)$ of a set of 20 inputs:

In [27]:
num_inputs = 20
data['s_4_1'] = 1

To generate the inputs, lets use a Gaussian with $\mu=50$ and $\sigma = 10$ (all input values rounded to integers):

In [28]:
input_vals = generate_variance_circuit.generate_inputs(num_inputs,50,10)
inputs_file = benchmark_utils.write_inputs_file(input_vals)

In [ ]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"uint32_t","HElib_F2",num_inputs,"debugvariance.txt")

In [5]:
results

{'Nxbar': '1001', 'varianceN3': '637980'}

In the clear, we can then divide these by $N$ and $N^3$ respectively:

In [6]:
print(float(results["Nxbar"])/num_inputs , float(results["varianceN3"])/pow(num_inputs,3))

50.05 79.7475


## Bitonic sort

Sorting a list of inputs is another non-trivial operation that can be performed using a combination of straightforward homomorphic operations - namely "Select" and "Compare".


In [2]:
circuit_file = generate_bitonic_sort_circuit.generate(4)

What inputs does this circuit expect?

In [3]:
benchmark_utils.get_inputs(circuit_file)

['i0', 'i1', 'i2', 'i3']

In [4]:
inputs_file = benchmark_utils.write_inputs_file({"i0":4,"i1":8,"i2":1,"i3": 3})

In [5]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"int8_t","HElib_F2")

In [6]:
results

{'w31': '1', 'w32': '3', 'w36': '4', 'w37': '8'}

In [7]:
results_tfhe = benchmark_utils.run_circuit(circuit_file,inputs_file,"int8_t","TFHE")

In [10]:
results_tfhe

{'w31': '1', 'w32': '3', 'w36': '4', 'w37': '8'}

In [8]:
import pandas as pd
all_rows = pd.read_sql(session.query(BenchmarkMeasurement).filter(BenchmarkMeasurement.circuit_name=="bitonic-sort").statement,session.bind)

In [9]:
all_rows[["context_name","input_bitwidth","circuit_name","execution_time","num_inputs"]]

,context_name,input_bitwidth,circuit_name,execution_time,num_inputs
0,HElib_F2,8,bitonic-sort,8.0,4
1,TFHE,8,bitonic-sort,10.0,4
